Backtesting QQQM, GOOG from 2/12 ~ 2/23, and compare the result with that operated by myself.

In [1]:
from datetime import datetime
import pytz
import pandas as pd
import yfinance as yf
import backtrader as bt
from backtrader_plotly.plotter import BacktraderPlotly
from backtrader_plotly.scheme import PlotScheme

In [2]:
# get QQQM and GOOG historical market data from 2024/2/12 to 2024/2/23
# qqqm = yf.Ticker("QQQM").history(start="2024-02-12", end="2024-02-23", interval="5m")
# goog = yf.Ticker("GOOG").history(start="2024-02-12", end="2024-02-23", interval="5m")
# qqqm.to_csv("../data/qqqm.csv"), goog.to_csv("../data/goog.csv")

In [3]:
qqqm = pd.read_csv("../data/qqqm.csv", index_col="Datetime", parse_dates=True)
goog = pd.read_csv("../data/goog.csv", index_col="Datetime", parse_dates=True)

## Grid Strategy

In [4]:
class GridStrategy(bt.Strategy):
    params = dict(
        long_bench=-0.01,
        short_bench=0.02,
    )

    lines = ("num_long",)

    def __init__(self):
        self.base_price = None

    def log(self, txt):
        print(f"[{self.data.datetime.datetime(0)}] {txt}")

    def next(self):
        # set current price as base price if not traded yet
        if not self.base_price:
            self.base_price = self.data.close[0]
            self.log(f"Setup base price as {self.base_price}")

        unit_value = self.broker.get_cash() / 10
        data_value = self.broker.get_value([self.data])
        returns = self.get_returns()

        # TODO: stop loss
        if self.position and self.get_returns(days=10) < -0.05:
            self.stop_loss()

        if self.position:
            if returns > 4 * self.p.short_bench and data_value > 0:
                self.order_target_value(target=0)
                self.base_price = self.data.close[0]
                self.log(
                    f"sell at grid 4, price: {self.base_price}, current value: {self.broker.get_value([self.data])}"
                )
            elif (
                returns > 3 * self.p.short_bench
                and data_value > 1 * unit_value
            ):
                self.order_target_value(target=1 * unit_value)
                self.base_price = self.data.close[0]
                self.log(
                    f"sell at grid 3, price:{self.base_price}, current value: {self.broker.get_value([self.data])}"
                )
            elif (
                returns > 2 * self.p.short_bench
                and data_value > 3 * unit_value
            ):
                self.order_target_value(target=3 * unit_value)
                self.base_price = self.data.close[0]
                self.log(
                    f"sell at grid 2, price:{self.base_price}, current value: {self.broker.get_value([self.data])}"
                )
            elif returns > self.p.short_bench and data_value > 6 * unit_value:
                self.order_target_value(target=6 * unit_value)
                self.base_price = self.data.close[0]
                self.log(
                    f"sell at grid 1, price:{self.base_price}, current value: {self.broker.get_value([self.data])}"
                )

        if self.broker.cash > 0:
            if (
                returns < 4 * self.p.long_bench
                and data_value < 10 * unit_value
            ):
                self.order_target_value(target=10 * unit_value)
                self.base_price = self.data.close[0]
                self.log(
                    f"buy at grid 4, price: {self.base_price}, current value: {self.broker.get_value([self.data])}"
                )
            elif (
                returns < 3 * self.p.long_bench and data_value < 9 * unit_value
            ):
                self.order_target_value(target=9 * unit_value)
                self.base_price = self.data.close[0]
                self.log(
                    f"buy at grid 3, price: {self.base_price}, current value: {self.broker.get_value([self.data])}"
                )
            elif (
                returns < 2 * self.p.long_bench and data_value < 7 * unit_value
            ):
                self.order_target_value(target=7 * unit_value)
                self.base_price = self.data.close[0]
                self.log(
                    f"buy at grid 2, price: {self.base_price}, current value: {self.broker.get_value([self.data])}"
                )
            elif returns < self.p.long_bench and data_value < 4 * unit_value:
                self.order_target_value(target=4 * unit_value)
                self.base_price = self.data.close[0]
                self.log(
                    f"buy at grid 1, price: {self.base_price}, current value: {self.broker.get_value([self.data])}"
                )

    def get_returns(self, days=5):
        return (self.data[0] - self.data[-days + 1]) / self.data[-days + 1]
        # return (self.data.close[0] - self.base_price) / self.base_price

    def stop_loss(self):
        self.log("liquidate position")
        self.log(f"Current value: {self.broker.get_value([self.data])}")
        self.order_target_value(target=0)
        self.base_price = self.data[0]
        self.log(f"set base price as {self.base_price}")

In [5]:
cerebro = bt.Cerebro()
cerebro.addwriter(bt.WriterFile)
cerebro.addstrategy(GridStrategy)

data = bt.feeds.PandasData(
    dataname=qqqm,
    timeframe=bt.TimeFrame.Minutes,
    compression=5,
)

cerebro.adddata(data)
result = cerebro.run()

cerebro.plot(
    iplot=False,
    #  Format string for the display of ticks on the x axis
    fmt_x_ticks="%Y-%b-%d %H:%M",
    # Format string for the display of data points values
    fmt_x_data="%Y-%b-%d %H:%M",
)

[2024-02-12 14:30:00] Setup base price as 180.0290069580078
[2024-02-13 14:30:00] buy at grid 1, price: 175.889892578125, current value: 0.0
[2024-02-22 14:30:00] sell at grid 1, price:178.63499450683594, current value: 3929.9698791503906
Cerebro:
  -----------------------------------------------------------------------------
  - Datas:
    +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
    - Data0:
      - Name: 
      - Timeframe: Minutes
      - Compression: 5
  -----------------------------------------------------------------------------
  - Strategies:
    +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
    - GridStrategy:
      *************************************************************************
      - Params:
        - long_bench: -0.01
        - short_bench: 0.02
      *************************************************************************
      - Indicators:
      *************************************************

[[<Figure size 2744x1484 with 4 Axes>]]